In [ ]:
# Install required packages
!pip install llama-cpp-python gradio langchain chromadb pypdf PyMuPDF sentence-transformers huggingface_hub


  Using cached llama_cpp_python-0.3.7.tar.gz (66.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached langchain-0.3.20-py3-none-any.whl.metadata (7.7 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached pymupdf-1.25.3-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached ffmpy-0.5.0-py3-none

# RAG with GUI to upload your doc and answer questions

In [ ]:
# RAG Application

import gradio as gr
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModel
import torch
import logging
import fitz  # PyMuPDF for PDF text extraction

# Setup Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load LLM model (Adjust path accordingly)
llm = Llama.from_pretrained(
        repo_id="dinukpathiraja/Qwen-2.5-3B-GRPO",
        filename="DataMavericks_Qwen_2.5_3B_GRPO.gguf",
        n_ctx=4096,  
        n_threads=4,   
        n_gpu_layers=-1
    )

# Load ColBERT model and tokenizer
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

# Device configuration for ColBERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
colbert_model = colbert_model.to(device)

# Initialize storage for embeddings and text chunks
file_embeddings = []
file_text_chunks = []

## File Processing Function (Supports .pdf, .txt, .md)
def process_file(uploaded_file):
    try:
        file_extension = uploaded_file.name.split('.')[-1].lower()

        # Extract text based on file type
        if file_extension == 'pdf':
            # Process PDF files using PyMuPDF
            doc = fitz.open(uploaded_file.name)
            extracted_text = ""
            for page_num in range(len(doc)):
                page = doc[page_num]
                extracted_text += page.get_text()
        elif file_extension in ['txt', 'md']:
            # Process plain text or markdown files
            with open(uploaded_file.name, 'r', encoding='utf-8') as f:
                extracted_text = f.read()
        else:
            return "❌ Unsupported file type. Please upload a .pdf, .txt, or .md file."

        # Clean non-UTF-8 characters from extracted text
        cleaned_text = extracted_text.encode("utf-8", "ignore").decode("utf-8", "ignore")

        # Split cleaned text into chunks for indexing
        splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
        texts = splitter.split_text(cleaned_text)

        # Index chunks into ColBERT or another vector database
        global file_embeddings, file_text_chunks
        file_embeddings.clear()
        file_text_chunks.clear()

        for chunk in texts:
            inputs = colbert_tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512).to(device)
            with torch.no_grad():
                embedding = colbert_model(**inputs).last_hidden_state.mean(dim=1)  # Average pooling
                file_embeddings.append(embedding.cpu())
                file_text_chunks.append(chunk)

        logging.info(f"Processed and indexed {len(texts)} chunks from the uploaded {file_extension.upper()} file.")
        print("PDF Processed Successfully")
        return f"✅ {file_extension.upper()} file processed successfully!"

    except Exception as e:
        logging.error(f"Error processing file: {e}")
        return f"❌ Error processing file: {str(e)}"


## Retrieve Context Using ColBERT
def get_context(question):
    try:
        # Encode the query using ColBERT
        inputs = colbert_tokenizer(question, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            query_embedding = colbert_model(**inputs).last_hidden_state.mean(dim=1).cpu()

        # Compute similarity scores between query and stored embeddings
        scores = [torch.cosine_similarity(query_embedding, emb, dim=1).item() for emb in file_embeddings]

        # Get top-3 most relevant chunks based on similarity scores
        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:3]
        top_chunks = [file_text_chunks[i] for i in top_indices]

        logging.info(f"Retrieved top-3 relevant chunks: {top_chunks}")
        
        return "\n".join(top_chunks) if top_chunks else "⚠️ No relevant context found."

    except Exception as e:
        logging.error(f"Error retrieving context: {e}")
        return "⚠️ Error retrieving context."


def answer_question(question, chat_history):
    logging.info(f"Received question: {question}")
    context = get_context(question)

    if not context or context == "⚠️ No relevant context found.":
        error_msg = "⚠️ No relevant context found."
        logging.warning(error_msg)
        chat_history.append((question, error_msg))
        return chat_history

    SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

    prompt = f"""<|system|>
{SYSTEM_PROMPT}

Context:
{context}

<|user|>
{question}
<|assistant|>
"""

    try:
        output = llm(
            prompt,
            max_tokens=1024,
            temperature=0.7,
            top_p=0.95,
            stop=["<|user|>", "<|system|>", "<|assistant|>"]
        )

        # Extract the full response text
        response_text = output['choices'][0]['text'].strip()
        print("Response:", response_text)

        # Parse reasoning and answer from the response
        reasoning_start = response_text.find("<reasoning>")
        reasoning_end = response_text.find("</reasoning>")
        answer_start = response_text.find("<answer>")
        answer_end = response_text.find("</answer>")

        reasoning = ""
        answer = ""

        if reasoning_start != -1 and reasoning_end != -1:
            reasoning = response_text[reasoning_start + len("<reasoning>"):reasoning_end].strip()

            # Check if <answer> tags exist properly
            if answer_start != -1 and answer_end != -1:
                answer = response_text[answer_start + len("<answer>"):answer_end].strip()
            else:
                # ELSE condition: <answer> tags not found correctly
                # Take everything after </reasoning>, remove any leftover tags
                remaining_text = response_text[reasoning_end + len("</reasoning>"):].strip()
                # Clean up any accidental tags
                remaining_text = remaining_text.replace("<answer>", "").replace("</answer>", "").strip()
                answer = remaining_text

        else:
            # If reasoning tags are missing entirely, treat whole text as answer
            reasoning = "Reasoning not explicitly provided."
            answer = response_text.replace("<answer>", "").replace("</answer>", "").strip()

        # Combine reasoning and answer for display
        formatted_response = f"**Reasoning:**\n{reasoning}\n\n**Answer:**\n{answer}"

        # Append to chat history
        chat_history.append((question, formatted_response))

    except Exception as e:
        error_msg = f"❌ Error generating response: {str(e)}"
        logging.error(error_msg)
        chat_history.append((question, error_msg))

    return chat_history



# Gradio UI (Continuous Chat Session with loading indicators)
with gr.Blocks() as demo:
    
    gr.Markdown("# 📄 Enhanced RAG Chatbot with Multi-format Support")

    with gr.Row():
        file_input = gr.File(label="Upload Document (.pdf/.txt/.md) 📁")
    
    upload_status = gr.Label("")
    upload_btn = gr.Button("Upload & Process File 🚀")

    upload_btn.click(
        fn=process_file,
        inputs=[file_input],
        outputs=[upload_status]
    )

    chatbot_ui = gr.Chatbot(label="Chat History 💬")
    
    question_input = gr.Textbox(label="Enter your question ❓")
    
    ask_btn = gr.Button("Get Answer ✨")

    ask_btn.click(
        fn=answer_question,
        inputs=[question_input, chatbot_ui],
        outputs=[chatbot_ui]
    )

demo.queue().launch()


c:\Users\USER\anaconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 26 key-value pairs and 434 tensors from C:\Users\USER\.cache\huggingface\hub\models--dinukpathiraja--Qwen-2.5-3B-GRPO\snapshots\35cb297a9213a3b775e597e357bae5a277f8cfd5\.\DataMavericks_Qwen_2.5_3B_GRPO.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 3b Instruct Unsloth Bnb 4bit
llama_model_loader: - kv   3:            

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2025-03-10 03:47:45,836 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-03-10 03:48:24,244 - INFO - Processed and indexed 30 chunks from the uploaded PDF file.


PDF Processed Successfully


2025-03-10 03:48:28,521 - INFO - Received question: What is Group Relative Policy Optimization 
2025-03-10 03:48:28,685 - INFO - Retrieved top-3 relevant chunks: ['R1-Zero can be further augmented through the application of majority voting. For example,\nwhen majority voting is employed on the AIME benchmark, DeepSeek-R1-Zero’s performance\nescalates from 71.0% to 86.7%, thereby exceeding the performance of OpenAI-o1-0912. The\nability of DeepSeek-R1-Zero to achieve such competitive performance, both with and without\nmajority voting, highlights its strong foundational capabilities and its potential for further\nadvancements in reasoning tasks.\nSelf-evolution Process of DeepSeek-R1-Zero\nThe self-evolution process of DeepSeek-R1-Zero\nis a fascinating demonstration of how RL can drive a model to improve its reasoning capabilities\nautonomously. By initiating RL directly from the base model, we can closely monitor the model’s\nprogression without the influence of the supervised fine-tu

Response: <reasoning>
Group Relative Policy Optimization (GRPO) is a reinforcement learning framework used in the context of this paper to improve the performance of the DeepSeek-R1-Zero model. It is not a separate concept, but rather a part of the larger RL process described in the document. GRPO is used to optimize the policy of the model, which in turn influences the model's decisions and actions during the RL process.
</reasoning>
<answer>
Group Relative Policy Optimization (GRPO) is a reinforcement learning framework used in the context of this paper to improve the performance of the DeepSeek-R1-Zero model. </answer>


2025-03-10 03:50:29,938 - INFO - Received question: Can you give me the formula for GRPO using the document
2025-03-10 03:50:30,130 - INFO - Retrieved top-3 relevant chunks: ['(2) DeepSeek-R1, which applies RL starting from a checkpoint fine-tuned with thousands of\nlong Chain-of-Thought (CoT) examples. 3) Distill the reasoning capability from DeepSeek-R1 to\nsmall dense models.\n2.2. DeepSeek-R1-Zero: Reinforcement Learning on the Base Model\nReinforcement learning has demonstrated significant effectiveness in reasoning tasks, as ev-\nidenced by our previous works (Shao et al., 2024; Wang et al., 2023). However, these works\nheavily depended on supervised data, which are time-intensive to gather. In this section, we\nexplore the potential of LLMs to develop reasoning capabilities without any supervised data,\nfocusing on their self-evolution through a pure reinforcement learning process. We start with a\nbrief overview of our RL algorithm, followed by the presentation of some exciting

Response: <reasoning>
The provided text doesn't give the exact formula for GRPO, but it does give the general structure and formula. The formula for the Group Relative Policy Optimization (GRPO) objective function is:

\[
J_{GRPPO}(\theta) = E_{q \sim P(Q), \{o_i\}_G \sim \pi_{\theta_0}(O|q)} \left[
\frac{1}{G} \sum_{i=1}^G \left( \min \left( \pi_{\theta}(o_i|q), \frac{\pi_{\theta_0}(o_i|q) \cdot \epsilon}{1 - \epsilon}, \frac{\pi_{\theta_0}(o_i|q) \cdot (1 + \epsilon)}{1 + \epsilon} \right) - \beta \cdot D_{KL}(\pi_\theta || \pi_{\ref}) \right) \cdot A_i
\right]
\]

Where:
- \( q \sim P(Q) \) represents sampling a question \( q \) from the distribution \( P(Q) \).
- \( \{o_i\}_G \sim \pi_{\theta_0}(O|q) \) represents sampling a group of outputs \( \{o_1, o_2, ..., o_G\} \) from the old policy \( \pi_{\theta_0} \).
- \( \pi_{\theta} \) is the policy model.
- \( \pi_{\theta_0} \) is the old policy.
- \( \epsilon \) and \( \beta \) are hyper-parameters.
- \( A_i \) is the advantage, calc

In [ ]:
!pip install ragas

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-webui 0.5.7 requires fastapi==0.111.0, but you have fastapi 0.115.8 which is incompatible.
open-webui 0.5.7 requires pydantic==2.9.2, but you have pydantic 2.10.6 which is incompatible.
open-webui 0.5.7 requires unstructured==0.15.9, but you have unstructured 0.16.23 which is incompatible.


2025-03-09 19:25:08,033 - INFO - Received question: Can you summerize the  DeepSeek-R1 Evaluation Percentages and Number Values
2025-03-09 19:25:08,103 - INFO - Retrieved top-3 relevant chunks: ['We intentionally limit our constraints to this structural format, avoiding any content-specific\nbiases—such as mandating reflective reasoning or promoting particular problem-solving strate-\ngies—to ensure that we can accurately observe the model’s natural progression during the RL\nprocess.\n2.2.4. Performance, Self-evolution Process and Aha Moment of DeepSeek-R1-Zero\nPerformance of DeepSeek-R1-Zero\nFigure 2 depicts the performance trajectory of DeepSeek-\nR1-Zero on the AIME 2024 benchmark throughout the RL training process. As illustrated,\nDeepSeek-R1-Zero demonstrates a steady and consistent enhancement in performance as the\nRL training advances. Notably, the average pass@1 score on AIME 2024 shows a significant\nincrease, jumping from an initial 15.6% to an impressive 71.0%, reaching

Response: <reasoning>
The question asks to summarize the evaluation percentages and number values for DeepSeek-R1. To answer it, I will look at the tables and figures provided in the text, and extract the relevant information.
</reasoning>
<answer>
From the text, we can see the following key information:

1. **AIME 2024 Benchmark:**
   - Initial Average pass@1 score: 15.6%
   - Final Average pass@1 score: 71.0%

2. **Other Benchmarks:**
   - **Math-500 Benchmark:**
     - OpenAI-o1-0912: 83.3%
     - DeepSeek-R1-Zero: 86.7%

   - **GPQA Benchmark:**
     - OpenAI-o1-0912: 94.8%
     - DeepSeek-R1-Zero: 95.9%

   - **LiveCode Benchmark:**
     - OpenAI-o1-0912: 77.3%
     - DeepSeek-R1-Zero: 73.3%

   - **Codeforces Benchmark:**
     - OpenAI-o1-0912: 63.8%
     - DeepSeek-R1-Zero: 1444 (Elo rating)

   - **Diamond Benchmark:**
     - OpenAI-o1-0912: 50.0%
     - DeepSeek-R1-Zero: 50.0%

   - **FRAMES Benchmark:**
     - OpenAI-o1-0912: 1843
     - DeepSeek-R1-Zero: 1444

   - **SimpleQ

In [25]:
import json
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import logging
import os 

# Define the process_file function
def process_file_from_path(file_path):
    """
    Processes a file based on its extension (.pdf, .txt, .md) and indexes its content into a vector database.
    
    Args:
        file_path (str): Path to the file to be processed.
    
    Returns:
        str: Success or error message.
    """
    try:
        # Check if the file exists
        if not os.path.exists(file_path):
            return f"❌ File not found: {file_path}"

        # Determine the file extension
        file_extension = file_path.split('.')[-1].lower()

        # Extract text based on file type
        if file_extension == 'pdf':
            # Process PDF files using PyMuPDF
            doc = fitz.open(file_path)
            extracted_text = ""
            for page_num in range(len(doc)):
                page = doc[page_num]
                extracted_text += page.get_text()
        elif file_extension in ['txt', 'md']:
            # Process plain text or markdown files
            with open(file_path, 'r', encoding='utf-8') as f:
                extracted_text = f.read()
        else:
            return "❌ Unsupported file type. Please upload a .pdf, .txt, or .md file."

        # Clean non-UTF-8 characters from extracted text
        cleaned_text = extracted_text.encode("utf-8", "ignore").decode("utf-8", "ignore")

        # Split cleaned text into chunks for indexing
        splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
        texts = splitter.split_text(cleaned_text)

        # Index chunks into ColBERT or another vector database
        global file_embeddings, file_text_chunks
        file_embeddings.clear()
        file_text_chunks.clear()

        for chunk in texts:
            inputs = colbert_tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512).to(device)
            with torch.no_grad():
                embedding = colbert_model(**inputs).last_hidden_state.mean(dim=1)  # Average pooling
                file_embeddings.append(embedding.cpu())
                file_text_chunks.append(chunk)

        logging.info(f"Processed and indexed {len(texts)} chunks from the uploaded {file_extension.upper()} file.")
        return f"✅ {file_extension.upper()} file processed successfully!"

    except Exception as e:
        logging.error(f"Error processing file: {e}")
        return f"❌ Error processing file: {str(e)}"

# Load JSON Dataset
def load_dataset(json_path):
    """
    Load the dataset from a JSON file.
    Args:
        json_path (str): Path to the JSON file containing questions and reference answers.
    Returns:
        list: Loaded dataset as a list of dictionaries.
    """
    with open(json_path, 'r') as f:
        return json.load(f)

# Define RAG System Function
def rag_system(question, chat_history):
    """
    Retrieves context using the RAG retrieval mechanism and generates an answer using the LLM.
    
    Args:
        question (str): The user-provided question.
        chat_history (list): Chat history for storing responses.
    
    Returns:
        str: The generated answer from the RAG system.
    """
    logging.info(f"Received question: {question}")
    context = get_context(question)  # Retrieve context using your retrieval mechanism

    if not context or context == "⚠️ No relevant context found.":
        error_msg = "⚠️ No relevant context found."
        logging.warning(error_msg)
        chat_history.append((question, error_msg))
        return error_msg

    SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

    prompt = f"""<|system|>
{SYSTEM_PROMPT}

Context:
{context}

<|user|>
{question}
<|assistant|>
"""

    try:
        output = llm(
            prompt,
            max_tokens=1024,
            temperature=0.7,
            top_p=0.95,
            stop=["<|user|>", "<|system|>", "<|assistant|>"]
        )

        # Extract the full response text
        response_text = output['choices'][0]['text'].strip()
        print("Response:", response_text)

        # Parse reasoning and answer from the response
        reasoning_start = response_text.find("<reasoning>")
        reasoning_end = response_text.find("</reasoning>")
        answer_start = response_text.find("<answer>")
        answer_end = response_text.find("</answer>")

        reasoning = ""
        answer = ""

        if reasoning_start != -1 and reasoning_end != -1:
            reasoning = response_text[reasoning_start + len("<reasoning>"):reasoning_end].strip()

            if answer_start != -1 and answer_end != -1:
                answer = response_text[answer_start + len("<answer>"):answer_end].strip()
            else:
                remaining_text = response_text[reasoning_end + len("</reasoning>"):].strip()
                remaining_text = remaining_text.replace("<answer>", "").replace("</answer>", "").strip()
                answer = remaining_text
        else:
            reasoning = "Reasoning not explicitly provided."
            answer = response_text.replace("<answer>", "").replace("</answer>", "").strip()

    except Exception as e:
        error_msg = f"❌ Error generating response: {str(e)}"
        logging.error(error_msg)
        chat_history.append((question, error_msg))
        return error_msg

    return answer

# Generate Answers Using RAG System
def generate_answers(rag_system, dataset):
    """
    Generate answers for each question in the dataset using the RAG system.
    Args:
        rag_system (function): Function to generate answers using the RAG system.
        dataset (list): List of dictionaries containing questions and reference answers.
    Returns:
        list: List of generated answers.
    """
    generated_answers = []
    chat_history = []
    
    for entry in dataset:
        question = entry["question"]
        generated_answer = rag_system(question, chat_history)
        generated_answers.append(generated_answer)
    
    return generated_answers

# Evaluate Generated Answers
def evaluate_answers(generated_answers, reference_answers):
    """
    Evaluate generated answers against reference answers using BLEU and ROUGE scores.
    
    Args:
        generated_answers (list): List of answers generated by the RAG system.
        reference_answers (list): List of reference answers from the dataset.
    
    Returns:
        dict: Dictionary containing BLEU and ROUGE scores for each answer pair.
    """
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    results = {"bleu_scores": [], "rouge_scores": []}

    for gen_answer, ref_answer in zip(generated_answers, reference_answers):
        # Calculate BLEU score
        bleu_score = sentence_bleu([ref_answer.split()], gen_answer.split())
        results["bleu_scores"].append(bleu_score)

        # Calculate ROUGE scores
        rouge_scores = rouge_scorer_obj.score(ref_answer, gen_answer)
        results["rouge_scores"].append(rouge_scores)

    return results

# Main Function for Evaluation
if __name__ == "__main__":
    # Path to JSON file containing questions and reference answers
    json_path = "D:\\Competitions\\39\\gguf\\qa_datasetmd.json"  # Replace with your file path
    file = process_file_from_path("D:\\Competitions\\39\\gguf\\dataset.md")
    if file:
        # Load dataset
        dataset = load_dataset(json_path)

        # Extract questions and reference answers
        questions = [entry["question"] for entry in dataset]
        reference_answers = [entry["reference_answer"] for entry in dataset]

        # Generate answers using RAG system
        print("Generating answers using the RAG system...")
        generated_answers = generate_answers(rag_system, dataset)

        # Evaluate generated answers against reference answers
        print("Evaluating generated answers...")
        evaluation_results = evaluate_answers(generated_answers, reference_answers)

        # Print Results
        print("\nEvaluation Results:")
        
        for i, (question, gen_answer, ref_answer) in enumerate(zip(questions, generated_answers, reference_answers)):
            print(f"Question {i+1}: {question}")
            print(f"Generated Answer: {gen_answer}")
            print(f"Reference Answer: {ref_answer}")
            print(f"BLEU Score: {evaluation_results['bleu_scores'][i]:.4f}")
            print(f"ROUGE Scores: {evaluation_results['rouge_scores'][i]}")


2025-03-09 20:09:11,321 - INFO - Processed and indexed 5 chunks from the uploaded MD file.
2025-03-09 20:09:11,323 - INFO - Received question: How does DualPipe optimize pipeline parallelism compared to 1F1B and ZB1P?
2025-03-09 20:09:11,347 - INFO - Retrieved top-3 relevant chunks: ['# DualPipe\nDualPipe is an innovative bidirectional pipeline parallelism algorithm introduced in the DeepSeek-V3 Technical Report. It achieves full overlap of forward and backward computation-communication phases, also reducing pipeline bubbles. For detailed information on computation-communication overlap, please refer to the profile data.\n\nPipeline Bubbles and Memory Usage Comparison\n\n| Method    | Bubble                  | Parameter | Activation |\n|:---------:|:-----------------------:|:---------:|:----------:|\n| 1F1B      | (PP-1)(𝐹+𝐵)            | 1×        | PP         |\n| ZB1P      | (PP-1)(𝐹+𝐵-2𝑊)         | 1×        | PP         |\n| DualPipe  | (PP/2-1)(𝐹&𝐵+𝐵-3𝑊)     | 2×        | PP+1   

Generating answers using the RAG system...


llama_perf_context_print:        load time =   77185.40 ms
llama_perf_context_print: prompt eval time =  388706.45 ms /  1609 tokens (  241.58 ms per token,     4.14 tokens per second)
llama_perf_context_print:        eval time =   55136.70 ms /   264 runs   (  208.85 ms per token,     4.79 tokens per second)
llama_perf_context_print:       total time =  101516.50 ms /  1873 tokens
2025-03-09 20:10:52,919 - INFO - Received question: What are the two expert load-balancing strategies in EPLB?
2025-03-09 20:10:52,994 - INFO - Retrieved top-3 relevant chunks: ["# Expert Parallelism Load Balancer (EPLB)\n\nWhen using expert parallelism (EP), different experts are assigned to different GPUs. Because the load of different experts may vary depending on the current workload, it is important to keep the load of different GPUs balanced. As described in the DeepSeek-V3 paper, we adopt a redundant experts strategy that duplicates heavy-loaded experts. Then, we heuristically pack the duplicated expe

Response: <reasoning>
To compare DualPipe with 1F1B (forward-backward) and ZB1P (Zero-Backward-1-Partition), we need to focus on the bubble reduction and overall performance improvements. The "Bubble" column in the table details the performance improvement of each method, which is a key metric for evaluating pipeline parallelism efficiency.

1F1B (Forward-Backward) and ZB1P (Zero-Backward-1-Partition) are both forward-backward pipeline parallelism methods, but they differ in how they handle the communication phases. 1F1B (Forward-Backward) and ZB1P (Zero-Backward-1-Partition) have different numbers of pipeline bubbles. 

DualPipe introduces an innovation by using an overlap of 2 forward and backward chunks, which reduces the number of pipeline bubbles compared to the other methods. 

<answer>
DualPipe optimizes pipeline parallelism by using an overlap of 2 forward and backward chunks, which results in fewer pipeline bubbles compared to 1F1B and ZB1P. This overlap allows for better util

llama_perf_context_print:        load time =   77185.40 ms
llama_perf_context_print: prompt eval time =   42840.11 ms /  1062 tokens (   40.34 ms per token,    24.79 tokens per second)
llama_perf_context_print:        eval time =   59078.96 ms /   272 runs   (  217.20 ms per token,     4.60 tokens per second)
llama_perf_context_print:       total time =  103124.48 ms /  1334 tokens
2025-03-09 20:12:36,152 - INFO - Received question: What architectural features make 3FS suitable for AI workloads?
2025-03-09 20:12:36,224 - INFO - Retrieved top-3 relevant chunks: ['### Global Load Balancing\n\nIn other cases, we use the global load balancing policy that replicates the experts globally regardless of expert groups, and pack the replicated experts to individual GPUs. This policy can be adopted in decoding stage with a larger expert-parallel size.\n\n# Fire-Flyer File system\nThe Fire-Flyer File System (3FS) is a high-performance distributed file system designed to address the challenges of A

...

llama_perf_context_print:        load time =   77185.40 ms
llama_perf_context_print: prompt eval time =   34992.54 ms /  1090 tokens (   32.10 ms per token,    31.15 tokens per second)
llama_perf_context_print:        eval time =   26253.23 ms /   175 runs   (  150.02 ms per token,     6.67 tokens per second)
llama_perf_context_print:       total time =   61777.28 ms /  1265 tokens
2025-03-09 20:16:34,915 - INFO - Received question: What GraySort benchmark results highlight 3FS's capabilities?
2025-03-09 20:16:34,992 - INFO - Retrieved top-3 relevant chunks: ['## Performance\n1. Peak throughput\n\nThe following figure demonstrates the throughput of read stress test on a large 3FS cluster. This cluster consists of 180 storage nodes, each equipped with 2×200Gbps InfiniBand NICs and sixteen 14TiB NVMe SSDs. Approximately 500+ client nodes were used for the read stress test, with each client node configured with 1x200Gbps InfiniBand NIC. The final aggregate read throughput reached approxim

Response: <reasoning>
The given context does not provide any information about 3FS (which seems to be a typo or misunderstanding, as the context is about DeepSeek-V3 and DualPipe), nor does it mention KVCache optimization for LLM inference. There is no data or information related to 3FS or KVCache optimization in the provided context. Therefore, based on the information given, it's impossible to answer the question about 3FS and KVCache optimization.
</reasoning>
<answer>
The information provided does not contain any details about 3FS or its performance in KVCache optimization for Large Language Models (LLMs) during inference. Since 3FS isn't mentioned in the context, and there's no data or information about KVCache optimization, it's not possible to answer this question based on the given context.
</answer>


llama_perf_context_print:        load time =   77185.40 ms
llama_perf_context_print: prompt eval time =   32910.38 ms /  1128 tokens (   29.18 ms per token,    34.27 tokens per second)
llama_perf_context_print:        eval time =   44383.15 ms /   300 runs   (  147.94 ms per token,     6.76 tokens per second)
llama_perf_context_print:       total time =   78335.24 ms /  1428 tokens
2025-03-09 20:17:53,343 - INFO - Using default tokenizer.


Response: <reasoning>
The GraySort benchmark results demonstrate 3FS's capabilities by showcasing how efficiently it can handle large-scale data sorting tasks. In this case, 3FS was able to achieve an average throughput of 3.66 TiB/min when sorting 110.5 TiB of data across 8,19gestions in 30 minutes and 14 seconds. This performance indicates that 3FS is capable of handling large datasets and sorting them efficiently, which is a crucial aspect for distributed systems that need to process large amounts of data in a timely manner. The ability to sort such a large dataset in a relatively short time demonstrates 3FS's strong performance and suitability for applications that require high throughput and strong consistency.</reasoning>
<answer>
The GraySort benchmark results highlight 3FS's capability by demonstrating that it can efficiently process and sort large-scale datasets. In this specific test, 3FS was able to achieve an average throughput of 3.66 TiB/min when sorting 110.5 TiB of data

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using 

In [31]:
import json
from ragas.metrics import (
    Faithfulness,
    AnswerRelevance,
    ContextRelevance,
    AnswerSimilarity,
    FactualCorrectness
)
from ragas.evaluation import evaluate_metrics
from ragas.samples import SingleTurnSample

# Define RAG System Function
def rag_system(question, chat_history):
    """
    Retrieves context using the RAG retrieval mechanism and generates an answer using the LLM.
    
    Args:
        question (str): The user-provided question.
        chat_history (list): Chat history for storing responses.
    
    Returns:
        dict: Generated answer and retrieved context.
    """
    logging.info(f"Received question: {question}")
    context = get_context(question)  # Retrieve context using your retrieval mechanism

    if not context or context == "⚠️ No relevant context found.":
        error_msg = "⚠️ No relevant context found."
        logging.warning(error_msg)
        chat_history.append((question, error_msg))
        return {"answer": error_msg, "context": ""}

    SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

    prompt = f"""<|system|>
{SYSTEM_PROMPT}

Context:
{context}

<|user|>
{question}
<|assistant|>
"""

    try:
        output = llm(
            prompt,
            max_tokens=1024,
            temperature=0.7,
            top_p=0.95,
            stop=["<|user|>", "<|system|>", "<|assistant|>"]
        )

        # Extract reasoning and answer from response
        response_text = output['choices'][0]['text'].strip()
        reasoning_start = response_text.find("<reasoning>")
        reasoning_end = response_text.find("</reasoning>")
        answer_start = response_text.find("<answer>")
        answer_end = response_text.find("</answer>")

        reasoning = ""
        answer = ""

        if reasoning_start != -1 and reasoning_end != -1:
            reasoning = response_text[reasoning_start + len("<reasoning>"):reasoning_end].strip()

            if answer_start != -1 and answer_end != -1:
                answer = response_text[answer_start + len("<answer>"):answer_end].strip()
            else:
                remaining_text = response_text[reasoning_end + len("</reasoning>"):].strip()
                remaining_text = remaining_text.replace("<answer>", "").replace("</answer>", "").strip()
                answer = remaining_text
        else:
            reasoning = "Reasoning not explicitly provided."
            answer = response_text.replace("<answer>", "").replace("</answer>", "").strip()

    except Exception as e:
        error_msg = f"❌ Error generating response: {str(e)}"
        logging.error(error_msg)
        chat_history.append((question, error_msg))
        return {"answer": error_msg, "context": ""}

    return {"answer": answer, "context": context}

# Generate Answers Using RAG System
def generate_answers(rag_system, dataset):
    """
    Generate answers for each question in the dataset using the RAG system.
    
    Args:
        rag_system (function): Function to generate answers using the RAG system.
        dataset (list): List of dictionaries containing questions and reference answers.
    
    Returns:
        list: List of dictionaries containing generated answers and contexts.
    """
    generated_data = []
    chat_history = []
    
    for entry in dataset:
        question = entry["question"]
        result = rag_system(question, chat_history)
        
        generated_data.append({
            "question": question,
            "generated_answer": result["answer"],
            "retrieved_context": result["context"],
            "reference_answer": entry.get("reference_answer", "")
        })
    
    return generated_data

# Evaluate Answers Using RAGAS Metrics
def evaluate_ragas(generated_data):
    """
    Evaluate generated answers against reference answers using RAGAS metrics.
    
    Args:
        generated_data (list): List of dictionaries containing questions, generated answers, contexts, and reference answers.
    
    Returns:
        dict: Dictionary containing scores for all RAGAS metrics.
    """
    samples = []
    
    for data in generated_data:
        sample = SingleTurnSample(
            question=data["question"],
            retrieved_context=data["retrieved_context"],
            ground_truth=data["reference_answer"],
            prediction=data["generated_answer"]
        )
        samples.append(sample)

    # Define metrics to evaluate
    metrics = {
        "Faithfulness": Faithfulness(),
        "Answer Relevance": AnswerRelevance(),
        "Context Relevance": ContextRelevance(),
        "Answer Similarity": AnswerSimilarity(),
        "Factual Correctness": FactualCorrectness()
    }

    # Evaluate metrics for all samples
    results = evaluate_metrics(samples, metrics)
    
    return results

# Main Function for Evaluation
if __name__ == "__main__":
    # Path to JSON file containing questions and reference answers
    json_path = "D:\\Competitions\\39\\gguf\\qa_datasetmd.json"  # Replace with your file path
    file = process_file_from_path("D:\\Competitions\\39\\gguf\\dataset.md")

    # Load dataset
    with open(json_path, 'r') as f:
        dataset = json.load(f)

    # Generate answers using RAG system
    print("Generating answers using the RAG system...")
    generated_data = generate_answers(rag_system, dataset)

    # Evaluate generated answers using RAGAS metrics
    print("Evaluating generated answers...")
    evaluation_results = evaluate_ragas(generated_data)

    # Print Results
    print("\nEvaluation Results:")
    
    for metric_name, score in evaluation_results.items():
        print(f"{metric_name}: {score:.4f}")


ModuleNotFoundError: No module named 'langchain_core.language_models'